## 4.1 Aussagekräftige Merkmale (Features)

In meinem ursprünglichen Modell habe ich nur den `Price` (Preis) verwendet. 
Für die Evaluation berücksichtige ich zusätzlich folgende Felder:
- `Positive` (Anzahl positiver Bewertungen): Ein Spiel mit vielen positiven Bewertungen wird tendenziell länger gespielt.
- `DLC count` (Anzahl DLCs): Mehr Inhalte könnten zu längerer Spielzeit führen.


In [15]:
import pandas as pd

# Daten laden
games = pd.read_csv(r"C:\Users\david\OneDrive\Desktop\Python lb\games (1).csv", delimiter=';', encoding='ISO-8859-1')

# Nur Spiele mit Spielzeit > 0
df = games[games['Average playtime forever'] > 0].copy()

# Korrelationen anzeigen
print("Korrelationen mit Spielzeit:")
print(df[['Price', 'Positive', 'DLC count', 'Average playtime forever']].corr()['Average playtime forever'])




Korrelationen mit Spielzeit:
Price                       0.090815
Positive                    0.193330
DLC count                   0.029058
Average playtime forever    1.000000
Name: Average playtime forever, dtype: float64


## 4.2 Messmethode: MSE (Mean Squared Error)

Zur Bewertung des Regressionsmodells wird der mittlere quadratische Fehler (MSE) verwendet. 
Der MSE misst die durchschnittliche quadratische Abweichung zwischen den tatsächlichen und den vorhergesagten Werten. 
Je kleiner der MSE, desto besser passt das Modell zu den Daten.


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# CSV einlesen
games = pd.read_csv(r"C:\Users\david\OneDrive\Desktop\Python lb\games (1).csv", delimiter=";", encoding="ISO-8859-1")

# Nur relevante Zeilen behalten
games = games.dropna(subset=["Average playtime forever", "Price", "Positive"])

# Eingangs und Zielvariablen definieren
X = games[["Price", "Positive"]]
y = games["Average playtime forever"]

# Trainings und Testdaten splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell aufbauen und trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen erstellen
predictions = model.predict(X_test)


In [16]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
print(" Mittlerer quadratischer Fehler (MSE):", round(mse, 2))


 Mittlerer quadratischer Fehler (MSE): 639355.76


## 4.3 Klassifikation + Wahrheitsmatrix

Um eine Klassifikation zu ermöglichen, wird ein Schwellenwert gesetzt:
- Wenn `Average playtime forever` > 500 Minuten → **Klasse 1** (viel gespielt)
- Ansonsten → **Klasse 0** (wenig gespielt)

Auf Basis dieser Klassen berechne ich die Wahrheitsmatrix.


In [13]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Schwelle definieren
threshold = 100

# Binäre Klassen erstellen
y_test_bin = (y_test >= threshold).astype(int)
predictions_bin = (predictions >= threshold).astype(int)

# Confusion Matrix berechnen
cm = confusion_matrix(y_test_bin, predictions_bin)
tn, fp, fn, tp = cm.ravel()

print(" Confusion Matrix:")
print(cm)
print(f"TN = {tn}, FP = {fp}, FN = {fn}, TP = {tp}")

# Precision & Recall berechnen
precision = precision_score(y_test_bin, predictions_bin)
recall = recall_score(y_test_bin, predictions_bin)

print(" Precision (Langspieler-Vorhersagen genauigkeit):", round(precision, 2))
print(" Recall (Wie viele Langspieler wurden erkannt):", round(recall, 2))


 Confusion Matrix:
[[10427  4487]
 [  755  1352]]
TN = 10427, FP = 4487, FN = 755, TP = 1352
 Precision (Langspieler-Vorhersagen genauigkeit): 0.23
 Recall (Wie viele Langspieler wurden erkannt): 0.64


## 4.4 Reflexion 

Das trainierte Modell liefert insgesamt brauchbare Vorhersagen zur Spielzeit basierend auf dem Preis und der Anzahl positiver Bewertungen. Der mittlere quadratische Fehler (MSE) liegt bei 639355.76, was auf gewisse Abweichungen hindeutet. Die Precision beträgt 0.23, der Recall 0.64, was zeigt, dass das Modell viele Langspieler erkennt, aber oft auch falsch liegt. Ursprünglich habe ich zusätzlich die Anzahl der DLCs analysiert, sie jedoch nicht ins finale Modell aufgenommen, da sie kaum Einfluss zeigten. Insgesamt lässt sich sagen, dass weitere Features wie Genre oder Reviews hilfreich wären.


